##Task 1

In [ ]:
import random
import math

class VLSI_Board:
    def __init__(self):

        self.board_width = 25
        self.board_height = 25


        self.blocks = {
            "ALU": [5, 5],
            "Cache": [7, 4],
            "Control Unit": [4, 4],
            "Register File": [6, 6],
            "Decoder": [5, 3],
            "Floating Unit": [5, 5]
        }


        self.connections = {
            "connection_1": ["Register File", "ALU"],
            "connection_2": ["Control Unit", "ALU"],
            "connection_3": ["ALU", "Cache"],
            "connection_4": ["Register File", "Floating Unit"],
            "connection_5": ["Cache", "Decoder"],
            "connection_6": ["Decoder", "Floating Unit"]
        }

        # Fitness function parameters
        self.alpha = 1000
        self.beta = 2
        self.gamma = 1

    def Eucledian_Distance(self, point1, point2):
        """Calculate the Euclidean distance between two points."""
        return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    def Wiring_Distance(self, chromosome):
        """Calculate the total wiring distance for the chromosome."""
        sum_distance = 0
        for c_name, c_blocks in self.connections.items():
            block1, block2 = c_blocks[0], c_blocks[1]
            point1 = chromosome[block1]
            point2 = chromosome[block2]
            sum_distance += self.Eucledian_Distance(point1, point2)
        return sum_distance

    def Bounding_Area(self, chromosome):


        x_coordinates = []
        y_coordinates = []

        for pos in chromosome.values():
            x_coordinates.append(pos[0])
            y_coordinates.append(pos[1])

        x_min = min(x_coordinates)
        y_min = min(y_coordinates)

        x_max_values = []
        y_max_values = []

        for block, pos in chromosome.items():

            right_edge = pos[0] + self.blocks[block][0]
            x_max_values.append(right_edge)


            top_edge = pos[1] + self.blocks[block][1]
            y_max_values.append(top_edge)

        x_max = max(x_max_values)
        y_max = max(y_max_values)

        return (x_max - x_min) * (y_max - y_min)

    def Overlap(self, pos1, pos2, dims1, dims2):

        pos1_right = pos1[0] + dims1[0]
        pos1_top = pos1[1] + dims1[1]
        pos2_right = pos2[0] + dims2[0]
        pos2_top = pos2[1] + dims2[1]

        # Check for overlap
        no_overlap = (pos1_right <= pos2[0] or pos1[0] >= pos2_right or
                     pos1_top <= pos2[1] or pos1[1] >= pos2_top)
        return not no_overlap

    def Total_Overlap(self, chromosome):

        O_count = 0
        block_list = list(chromosome.items())

        for i in range(len(block_list)):
            for j in range(i + 1, len(block_list)):
                block1, pos1 = block_list[i]
                block2, pos2 = block_list[j]
                dims1 = self.blocks[block1]
                dims2 = self.blocks[block2]

                if self.Overlap(pos1, pos2, dims1, dims2):
                    O_count += 1

        return O_count

    def Fitness(self, chromosome):

        overlap_points = self.Total_Overlap(chromosome)
        wiring_distance = self.Wiring_Distance(chromosome)
        BA = self.Bounding_Area(chromosome)

        #  sample: -(alpha * overlap + beta * wiring + gamma * bounding)
        F_value = -(self.alpha * overlap_points + self.beta * wiring_distance + self.gamma * BA)
        return F_value

    def initialize_population(self, size=6):

        population = []

        for k in range(size):
            chromosome = {}
            for block in self.blocks:
                x = random.randint(0, self.board_width - self.blocks[block][0])
                y = random.randint(0, self.board_height - self.blocks[block][1])
                chromosome[block] = [x, y]
            population.append(chromosome)

        return population

    def Sorted_Fitness(self, population):

        # list of (fitness, chromosome) pairs
        F_pairs = []
        for chromosome in population:
            F_value = self.Fitness(chromosome)
            F_pairs.append([F_value, chromosome])

        # Bubble sort
        n = len(F_pairs)
        for i in range(n):
            for j in range(0, n - i - 1):
                if F_pairs[j][0] > F_pairs[j + 1][0]:
                    F_pairs[j], F_pairs[j + 1] = F_pairs[j + 1], F_pairs[j]

        sorted_population = []
        for pair in F_pairs:
            sorted_population.append(pair[1])

        return sorted_population

    def Parent_Selection(self, population):
        return random.sample(population, 2)

    def Crossover(self, parent1, parent2):
        #Single-point crossover
        block_names = list(self.blocks.keys())
        split_point = random.randint(1, len(self.blocks) - 1)

        child1 = {}
        child2 = {}

        for i in range(split_point):
            block_name = block_names[i]
            child1[block_name] = parent1[block_name][:]
            child2[block_name] = parent2[block_name][:]  #List Copy

        for i in range(split_point, len(self.blocks)):
            block_name = block_names[i]
            child1[block_name] = parent2[block_name][:]
            child2[block_name] = parent1[block_name][:]  #List Copy

        return child1, child2

    def Mutation(self, chromosome, mutation_rate=0.1):
        #According to the question
        if mutation_rate < 0.05:
            mutation_rate = 0.05
        elif mutation_rate > 0.10:
            mutation_rate = 0.10

        if random.random() < mutation_rate:
            block = random.choice(list(self.blocks.keys()))
            x = random.randint(0, self.board_width - self.blocks[block][0])
            y = random.randint(0, self.board_height - self.blocks[block][1])
            chromosome[block] = [x, y]

    def Fitness_Compare(self, chrom1, chrom2):

        fitness1 = self.Fitness(chrom1)
        fitness2 = self.Fitness(chrom2)
        return fitness1 > fitness2

    def sort_population_manual(self, population):

        n = len(population)
        for i in range(n):
            for j in range(0, n - i - 1):
                if not self.Fitness_Compare(population[j], population[j + 1]):
                    population[j], population[j + 1] = population[j + 1], population[j]
        return population

    def Genetic_Algo(self, generations=15):

        population = self.initialize_population()
        best_chromosome = None
        best_fitness = float('-inf')  # Negative Infinity

        for generation in range(generations):
            population = self.sort_population_manual(population)

            current_best_fitness = self.Fitness(population[0])
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_chromosome = population[0].copy()

            print(f"Generation {generation + 1}: Best fitness = {current_best_fitness}")


            elite_count = 2
            new_population = population[:elite_count]

            # Offspring
            while len(new_population) < len(population):
                parent1, parent2 = self.Parent_Selection(population)
                child1, child2 = self.Crossover(parent1, parent2)


                self.Mutation(child1, mutation_rate=0.07)
                self.Mutation(child2, mutation_rate=0.07)

                new_population.extend([child1, child2])


            population = new_population[:len(population)]

        print(f"Total generations processed: {generations}")
        return best_chromosome, best_fitness

# Testing
def final():
    GA = VLSI_Board()
    best_layout, best_F_value = GA.Genetic_Algo()


    O_count = GA.Total_Overlap(best_layout)
    wiring_distance = GA.Wiring_Distance(best_layout)
    bounding_area = GA.Bounding_Area(best_layout)

    print(f"\n")
    print(f"Best total fitness value: {best_F_value}")
    print(f"Total wiring length: {wiring_distance}")
    print(f"Total bounding box area: {bounding_area}")
    print(f"Total overlap counts: {O_count}")


    print(f"\n")
    block_order = ["ALU", "Cache", "Control Unit", "Register File", "Decoder", "Floating Unit"]
    for block in block_order:
        if block in best_layout:
            x, y = best_layout[block]
            print(f"{block}: ({x}, {y})")

# Algorithm run
final()

Generation 1: Best fitness = -635.3690936098045
Generation 2: Best fitness = -635.3690936098045
Generation 3: Best fitness = -635.3690936098045
Generation 4: Best fitness = -635.3690936098045
Generation 5: Best fitness = -586.8265669607838
Generation 6: Best fitness = -586.8265669607838
Generation 7: Best fitness = -586.8265669607838
Generation 8: Best fitness = -586.8265669607838
Generation 9: Best fitness = -586.8265669607838
Generation 10: Best fitness = -586.8265669607838
Generation 11: Best fitness = -586.8265669607838
Generation 12: Best fitness = -586.8265669607838
Generation 13: Best fitness = -586.8265669607838
Generation 14: Best fitness = -586.8265669607838
Generation 15: Best fitness = -586.8265669607838
Total generations processed: 15


Best total fitness value: -586.8265669607838
Total wiring length: 68.41328348039187
Total bounding box area: 450
Total overlap counts: 0


ALU: (18, 20)
Cache: (18, 12)
Control Unit: (9, 19)
Register File: (7, 9)
Decoder: (8, 16)
Floating U

##Test 2

In [ ]:
import random
import math

class VLSI_Board:
    def __init__(self):

        self.board_width = 25
        self.board_height = 25


        self.blocks = {
            "ALU": [5, 5],
            "Cache": [7, 4],
            "Control Unit": [4, 4],
            "Register File": [6, 6],
            "Decoder": [5, 3],
            "Floating Unit": [5, 5]
        }


        self.connections = {
            "connection_1": ["Register File", "ALU"],
            "connection_2": ["Control Unit", "ALU"],
            "connection_3": ["ALU", "Cache"],
            "connection_4": ["Register File", "Floating Unit"],
            "connection_5": ["Cache", "Decoder"],
            "connection_6": ["Decoder", "Floating Unit"]
        }

        # Fitness function parameters
        self.alpha = 1000
        self.beta = 2
        self.gamma = 1

    def Eucledian_Distance(self, point1, point2):
        """Calculate the Euclidean distance between two points."""
        return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    def Wiring_Distance(self, chromosome):
        """Calculate the total wiring distance for the chromosome."""
        sum_distance = 0
        for c_name, c_blocks in self.connections.items():
            block1, block2 = c_blocks[0], c_blocks[1]
            point1 = chromosome[block1]
            point2 = chromosome[block2]
            sum_distance += self.Eucledian_Distance(point1, point2)
        return sum_distance

    def Bounding_Area(self, chromosome):


        x_coordinates = []
        y_coordinates = []

        for pos in chromosome.values():
            x_coordinates.append(pos[0])
            y_coordinates.append(pos[1])

        x_min = min(x_coordinates)
        y_min = min(y_coordinates)

        x_max_values = []
        y_max_values = []

        for block, pos in chromosome.items():

            right_edge = pos[0] + self.blocks[block][0]
            x_max_values.append(right_edge)


            top_edge = pos[1] + self.blocks[block][1]
            y_max_values.append(top_edge)

        x_max = max(x_max_values)
        y_max = max(y_max_values)

        return (x_max - x_min) * (y_max - y_min)

    def Overlap(self, pos1, pos2, dims1, dims2):

        pos1_right = pos1[0] + dims1[0]
        pos1_top = pos1[1] + dims1[1]
        pos2_right = pos2[0] + dims2[0]
        pos2_top = pos2[1] + dims2[1]

        # Check for overlap
        no_overlap = (pos1_right <= pos2[0] or pos1[0] >= pos2_right or
                     pos1_top <= pos2[1] or pos1[1] >= pos2_top)
        return not no_overlap

    def Total_Overlap(self, chromosome):

        O_count = 0
        block_list = list(chromosome.items())

        for i in range(len(block_list)):
            for j in range(i + 1, len(block_list)):
                block1, pos1 = block_list[i]
                block2, pos2 = block_list[j]
                dims1 = self.blocks[block1]
                dims2 = self.blocks[block2]

                if self.Overlap(pos1, pos2, dims1, dims2):
                    O_count += 1

        return O_count

    def Fitness(self, chromosome):

        overlap_points = self.Total_Overlap(chromosome)
        wiring_distance = self.Wiring_Distance(chromosome)
        BA = self.Bounding_Area(chromosome)

        #  sample: -(alpha * overlap + beta * wiring + gamma * bounding)
        F_value = -(self.alpha * overlap_points + self.beta * wiring_distance + self.gamma * BA)
        return F_value

    def initialize_population(self, size=6):

        population = []

        for k in range(size):
            chromosome = {}
            for block in self.blocks:
                x = random.randint(0, self.board_width - self.blocks[block][0])
                y = random.randint(0, self.board_height - self.blocks[block][1])
                chromosome[block] = [x, y]
            population.append(chromosome)

        return population

    def Sorted_Fitness(self, population):

        # list of (fitness, chromosome) pairs
        F_pairs = []
        for chromosome in population:
            F_value = self.Fitness(chromosome)
            F_pairs.append([F_value, chromosome])

        # Bubble sort
        n = len(F_pairs)
        for i in range(n):
            for j in range(0, n - i - 1):
                if F_pairs[j][0] > F_pairs[j + 1][0]:
                    F_pairs[j], F_pairs[j + 1] = F_pairs[j + 1], F_pairs[j]

        sorted_population = []
        for pair in F_pairs:
            sorted_population.append(pair[1])

        return sorted_population

    def Parent_Selection(self, population):
        return random.sample(population, 2)

    def Crossover(self, parent1, parent2):
      # Two-point Crossover
        block_names = list(self.blocks.keys())
        num_blocks = len(block_names)

        # Randomly select two crossover points
        # Ensure point1 < point2 and both are within valid range
        point1 = random.randint(1, num_blocks - 2)
        point2 = random.randint(point1 + 1, num_blocks - 1)

        child1 = {}
        child2 = {}


        for i in range(point1):
            block_name = block_names[i]
            child1[block_name] = parent1[block_name][:]
            child2[block_name] = parent2[block_name][:]  #List Copy


        for i in range(point1, point2):
            block_name = block_names[i]
            child1[block_name] = parent2[block_name][:]
            child2[block_name] = parent1[block_name][:]  #List Copy


        for i in range(point2, num_blocks):
            block_name = block_names[i]
            child1[block_name] = parent1[block_name][:]
            child2[block_name] = parent2[block_name][:]  #List Copy

        return child1, child2



    def Mutation(self, chromosome, mutation_rate=0.1):
        #According to the question
        if mutation_rate < 0.05:
            mutation_rate = 0.05
        elif mutation_rate > 0.10:
            mutation_rate = 0.10

        if random.random() < mutation_rate:
            block = random.choice(list(self.blocks.keys()))
            x = random.randint(0, self.board_width - self.blocks[block][0])
            y = random.randint(0, self.board_height - self.blocks[block][1])
            chromosome[block] = [x, y]

    def Fitness_Compare(self, chrom1, chrom2):

        fitness1 = self.Fitness(chrom1)
        fitness2 = self.Fitness(chrom2)
        return fitness1 > fitness2

    def sort_population_manual(self, population):

        n = len(population)
        for i in range(n):
            for j in range(0, n - i - 1):
                if not self.Fitness_Compare(population[j], population[j + 1]):
                    population[j], population[j + 1] = population[j + 1], population[j]
        return population

    def Genetic_Algo(self, generations=15):

        population = self.initialize_population()
        best_chromosome = None
        best_fitness = float('-inf')  # Negative Infinity

        for generation in range(generations):
            population = self.sort_population_manual(population)

            current_best_fitness = self.Fitness(population[0])
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_chromosome = population[0].copy()

            print(f"Generation {generation + 1}: Best fitness = {current_best_fitness}")


            elite_count = 2
            new_population = population[:elite_count]

            # Offspring
            while len(new_population) < len(population):
                parent1, parent2 = self.Parent_Selection(population)
                child1, child2 = self.Crossover(parent1, parent2)


                self.Mutation(child1, mutation_rate=0.07)
                self.Mutation(child2, mutation_rate=0.07)

                new_population.extend([child1, child2])


            population = new_population[:len(population)]

        print(f"Total generations processed: {generations}")
        return best_chromosome, best_fitness

# Testing
def final():

    GA = VLSI_Board()
    best_layout, best_F_value = GA.Genetic_Algo()


    O_count = GA.Total_Overlap(best_layout)
    wiring_distance = GA.Wiring_Distance(best_layout)
    bounding_area = GA.Bounding_Area(best_layout)

    print(f"\n")
    print(f"Best total fitness value: {best_F_value}")
    print(f"Total wiring length: {wiring_distance}")
    print(f"Total bounding box area: {bounding_area}")
    print(f"Total overlap counts: {O_count}")


    print(f"\n")
    block_order = ["ALU", "Cache", "Control Unit", "Register File", "Decoder", "Floating Unit"]
    for block in block_order:
        if block in best_layout:
            x, y = best_layout[block]
            print(f"{block}: ({x}, {y})")

# Algorithm run
final()

Generation 1: Best fitness = -1552.6072853529163
Generation 2: Best fitness = -616.4699875657733
Generation 3: Best fitness = -616.4699875657733
Generation 4: Best fitness = -616.4699875657733
Generation 5: Best fitness = -499.51293054003213
Generation 6: Best fitness = -499.51293054003213
Generation 7: Best fitness = -499.51293054003213
Generation 8: Best fitness = -499.51293054003213
Generation 9: Best fitness = -499.51293054003213
Generation 10: Best fitness = -499.51293054003213
Generation 11: Best fitness = -499.51293054003213
Generation 12: Best fitness = -499.51293054003213
Generation 13: Best fitness = -499.51293054003213
Generation 14: Best fitness = -499.51293054003213
Generation 15: Best fitness = -499.51293054003213
Total generations processed: 15


Best total fitness value: -499.51293054003213
Total wiring length: 62.75646527001608
Total bounding box area: 374
Total overlap counts: 0


ALU: (3, 14)
Cache: (10, 3)
Control Unit: (2, 21)
Register File: (0, 7)
Decoder: (11, 19